In [3]:
# --- Бібліотеки для даної роботи ---
try:
    import numpy, pandas
    print("Бібліотеки вже встановлені. Пропускаємо інсталяцію.")
except ImportError:
    print("Встановлюємо бібліотеки...")
    %pip install numpy pandas

Бібліотеки вже встановлені. Пропускаємо інсталяцію.


## Задача 4. Прогноз тренду (Лінійна регресія)

**Умова:**

При моніторингу IT-інфраструктури дані часто надходять із шумом.

Навантаження на CPU сервера може коливатися, але загальний тренд вказує на поступове зростання.

Щоб передбачити, коли сервер досягне критичного навантаження, потрібно побудувати лінію тренду $y = kx + b$.

Оскільки точки не лежать ідеально на одній прямій, точного розв'язку системи не існує, тому використовується **метод найменших квадратів (МНК)**.

**Дано виміри завантаження CPU $y$ протягом 5 годин $t$:**
* Час $t = [1, 2, 3, 4, 5]$
* Навантаження $y = [22, 28, 37, 45, 53]$

**Використовуючи бібліотеку `numpy`, напишіть програму та виконайте наступні кроки:**
1.  Сформуйте матрицю $A$. Пам'ятайте, що для знаходження вільного члена $b$ матриця повинна містити стовпчик зі значень $t$ та стовпчик з одиниць.
2.  Знайдіть коефіцієнти прямої $k$ та $b$ за допомогою функції `np.linalg.lstsq(A, y)`. Ця функція повертає найкращий наближений розв'язок.
3.  Використовуючи знайдені коефіцієнти, обчисліть прогнозоване навантаження на **6-ту годину** за формулою $\hat{y} = k \cdot 6 + b$.
4.  Виведіть рівняння отриманого тренду та прогноз.

**Опціональне завдання з зірочкою (+5 бонусних балів):**
5. Функція `np.linalg.lstsq()` використовує нормальне рівняння для знаходження оптимальних коефіцієнтів. Реалізуйте цей метод вручну.

Знайдіть коефіцієнти $k$ - нахил, та $b$ - зсув через нормальне рівняння
$$A^T A \mathbf{x} = A^T \mathbf{y}$$
де $\mathbf{x} = [k, b]^T$ — вектор невідомих коефіцієнтів.

**Кроки:**
1.  Обчислити $A^T A$.
2.  Обчислити $A^T \mathbf{y}$.
3.  Розв'язати систему через `np.linalg.solve()`.
4.  Порівняйте результат з `lstsq` — вони мають збігатися

In [4]:
import numpy as np
import pandas as pd
from IPython.display import display

time_t = np.array([1, 2, 3, 4, 5])
load_y = np.array([22, 28, 37, 45, 53])

matrix_A = np.column_stack((time_t, np.ones_like(time_t)))

solution_lstsq, residuals, rank, s = np.linalg.lstsq(matrix_A, load_y, rcond=None)
k_auto, b_auto = solution_lstsq

matrix_A_T = matrix_A.T
matrix_ATA = np.dot(matrix_A_T, matrix_A)
vector_ATy = np.dot(matrix_A_T, load_y)

solution_manual = np.linalg.solve(matrix_ATA, vector_ATy)
k_manual, b_manual = solution_manual

forecast_time = 6
forecast_load = k_auto * forecast_time + b_auto

print("Красивий Вивід:")
print("Результати аналізу тренду:")

df_comparison = pd.DataFrame({
    "Метод": ["Автоматичний (lstsq)", "Ручний (Normal Eq)"],
    "Нахил (k)": [k_auto, k_manual],
    "Зсув (b)": [b_auto, b_manual],
    "Прогноз (t=6)": [k_auto * 6 + b_auto, k_manual * 6 + b_manual]
})

df_forecast = pd.DataFrame({
    "Параметр": ["Рівняння тренду", "Прогноз на 6-ту годину"],
    "Значення": [f"y = {k_auto:.2f}x + {b_auto:.2f}", f"{forecast_load:.2f}% CPU"]
})

display(df_comparison.style.background_gradient(subset=["Прогноз (t=6)"], cmap="Greens"))
print("\nПідсумок:")
display(df_forecast.style.hide(axis='index'))

print("Технічний вивід:")
print("-" * 50)
print(f"Матриця A (Design Matrix):\n{matrix_A}")
print(f"Матриця A.T * A:\n{matrix_ATA}")
print(f"Вектор A.T * y:\n{vector_ATy}")
print(f"Коефіцієнти (k, b) Авто: {solution_lstsq}")
print(f"Коефіцієнти (k, b) Вручну: {solution_manual}")
print(f"Прогноз на t=6: {forecast_load}")
print(f"Збіг методів: {np.allclose(solution_lstsq, solution_manual)}")

Красивий Вивід:
Результати аналізу тренду:


,Метод,Нахил (k),Зсув (b),Прогноз (t=6)
0,Автоматичний (lstsq),7.900000,13.300000,60.700000
1,Ручний (Normal Eq),7.900000,13.300000,60.700000



Підсумок:


Параметр,Значення
Рівняння тренду,y = 7.90x + 13.30
Прогноз на 6-ту годину,60.70% CPU


Технічний вивід:
--------------------------------------------------
Матриця A (Design Matrix):
[[1 1]
 [2 1]
 [3 1]
 [4 1]
 [5 1]]
Матриця A.T * A:
[[55 15]
 [15  5]]
Вектор A.T * y:
[634 185]
Коефіцієнти (k, b) Авто: [ 7.9 13.3]
Коефіцієнти (k, b) Вручну: [ 7.9 13.3]
Прогноз на t=6: 60.69999999999999
Збіг методів: True


## Аналітичний розв'язок (Ручний МНК)

Ми шукаємо параметри прямої $y = kx + b$.
Маємо систему рівнянь у матричному вигляді $A\mathbf{x} \approx \mathbf{y}$, де:

$$
A = \begin{pmatrix}
1 & 1 \\
2 & 1 \\
3 & 1 \\
4 & 1 \\
5 & 1
\end{pmatrix}, \quad
\mathbf{y} = \begin{pmatrix} 22 \\ 28 \\ 37 \\ 45 \\ 53 \end{pmatrix}, \quad
\mathbf{x} = \begin{pmatrix} k \\ b \end{pmatrix}
$$

Для знаходження найкращого наближення розв'язуємо **нормальне рівняння**:
$$A^T A \mathbf{x} = A^T \mathbf{y}$$

**Крок 1. Обчислюємо матрицю $A^T A$:**
Це матриця розміру $2 \times 2$, де елементи — це суми степенів $t$:
$$
A^T A = \begin{pmatrix}
\sum t^2 & \sum t \\
\sum t & n
\end{pmatrix} = \begin{pmatrix}
1+4+9+16+25 & 1+2+3+4+5 \\
1+2+3+4+5 & 5
\end{pmatrix} = \begin{pmatrix}
55 & 15 \\
15 & 5
\end{pmatrix}
$$

**Крок 2. Обчислюємо вектор $A^T \mathbf{y}$:**
$$
A^T \mathbf{y} = \begin{pmatrix}
\sum t \cdot y \\
\sum y
\end{pmatrix} = \begin{pmatrix}
1\cdot22 + 2\cdot28 + 3\cdot37 + 4\cdot45 + 5\cdot53 \\
22 + 28 + 37 + 45 + 53
\end{pmatrix} = \begin{pmatrix}
634 \\
185
\end{pmatrix}
$$

**Крок 3. Розв'язуємо систему лінійних рівнянь:**
$$
\begin{cases}
55k + 15b = 634 \\
15k + 5b = 185
\end{cases}
$$

Поділимо друге рівняння на 5:
$$3k + b = 37 \implies b = 37 - 3k$$

Підставимо у перше:
$$55k + 15(37 - 3k) = 634$$
$$55k + 555 - 45k = 634$$
$$10k = 79 \implies k = 7.9$$

Знайдемо $b$:
$$b = 37 - 3(7.9) = 37 - 23.7 = 13.3$$

**Рівняння тренду:**
$$y = 7.9x + 13.3$$

**Прогноз на 6-ту годину:**
$$y(6) = 7.9 \cdot 6 + 13.3 = 47.4 + 13.3 = \mathbf{60.7}$$